In [37]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

Done


# Changing Genres into individual Columns

In [38]:
# make a set with all unique genres

genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

{'Animation', 'Drama', 'Sport', 'Crime', 'Action', 'Family', 'Romance', 'Game-Show', 'History', 'Thriller', 'Sci-Fi', 'Horror', 'War', 'Musical', 'Mystery', 'Adventure', 'News', 'Fantasy', 'Reality-TV', 'Film-Noir', 'Biography', 'Documentary', 'Comedy', 'Short', 'Western', 'Music'}


In [39]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [40]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

movie_data_1 = movie_data.copy()
movie_data_2 = movie_data.copy()
movie_data_3 = movie_data.copy()
movie_data_4 = movie_data.copy()

movie_data_1 = remove_too_many_NaN(movie_data, 1)
movie_data_2 = remove_too_many_NaN(movie_data, 2)
movie_data_3 = remove_too_many_NaN(movie_data, 3)
movie_data_4 = remove_too_many_NaN(movie_data, 4)

Length before deletion: 5043
Length after deletion: 3771
Length before deletion: 5043
Length after deletion: 4490
Length before deletion: 5043
Length after deletion: 4777
Length before deletion: 5043
Length after deletion: 4900


# Changing Color to Numeric

In [41]:
def numeric_color(movie_data):
    #replaces the color value to 1
    movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
    movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

    #replaces the NaN or black and white values to 0
    for item in movie_data.color:
        if item != 1:
            movie_data.color = movie_data.color.replace(to_replace = item, value = 0)

    #movie_data.color = new_color_column 
    #print(movie_data['color'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None
    print('Done')
    return movie_data

movie_data_1 = numeric_color(movie_data_1)
movie_data_2 = numeric_color(movie_data_2)
movie_data_3 = numeric_color(movie_data_3)
movie_data_4 = numeric_color(movie_data_4)

Done
Done
Done
Done


# Changing Country to Numeric

In [42]:
def numeric_country(movie_data):
    #replaces the USA values to 1
    movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
    movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
    #replaces the NaN or non USA values to 0
    for item in movie_data.country:
        if item != 1:
            movie_data.country = movie_data.country.replace(to_replace = item, value = 0)

    #print(movie_data['country'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None

    print('Done')
    return movie_data

movie_data_1 = numeric_country(movie_data_1)
movie_data_2 = numeric_country(movie_data_2)
movie_data_3 = numeric_country(movie_data_3)
movie_data_4 = numeric_country(movie_data_4)

Done
Done
Done
Done


# Changing Languange to Numeric

In [43]:
def numeric_language(movie_data):
    #replaces the English values to 1
    movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
    movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

    #replaces the other values to 0
    for item in movie_data.language:
        if item != 1:
            movie_data.language = movie_data.language.replace(to_replace = item, value = 0)

    #print(movie_data['language'])

    #makes sure that there is nog error where it shouldn't be
    pd.options.mode.chained_assignment= None
    print('Done')
    return movie_data

movie_data_1 = numeric_language(movie_data_1)
movie_data_2 = numeric_language(movie_data_2)
movie_data_3 = numeric_language(movie_data_3)
movie_data_4 = numeric_language(movie_data_4)

Done
Done
Done
Done


# Chaning Content_Rating to Numeric

In [44]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data_1 = content_to_numerical(movie_data_1)
movie_data_2 = content_to_numerical(movie_data_2)
movie_data_3 = content_to_numerical(movie_data_3)
movie_data_4 = content_to_numerical(movie_data_4)

# Replacing NaNs with averages

In [45]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes','gross', 'title_year'
                   , 'duration', 'facenumber_in_poster', 'content_rating']

for column in to_replace_NaNs:
    movie_data_1[column] = replace_NaNs(movie_data_1[column])
    movie_data_2[column] = replace_NaNs(movie_data_2[column])
    movie_data_3[column] = replace_NaNs(movie_data_3[column])
    movie_data_4[column] = replace_NaNs(movie_data_4[column])


7722.74303898
7032.59888641
6747.99685995
6648.62979592
2014.86157518
1792.34922049
1713.49706868
1684.1759902
768.95332803
691.49108337
664.698322851
653.779595175
2.11111376293
2.1096122633
2.11774347826
2.1284344634
46074665.6696
41840819.2828
40444270.7169
39892924.8304
11486.9586317
10423.8097996
10003.1101109
9849.74408163
804.246353752
744.864587973
703.776847394
692.662510221
52410371.7385
49287524.5918
48619441.8755
48514997.6517
2003.01060727
2001.97906459
2002.19866025
2002.39901881
110.219570406
109.24142539
108.555276382
108.243568804
1.37708830549
1.35540691193
1.36479966436
1.36272504092
14.1148236542
13.7810398924
13.6992675571
13.6882617062


# Delete columns with too little 1's

In [46]:
def del_sparse_cols(movie_data, doprint=False):
    for genre in genres_set:
        if doprint:
            print(genre, ':')
        counter = 0
        for value in movie_data[genre]:
            if value == 1.0:
                counter+=1
        
        if doprint:
            print(counter)

    weghalen = ['Game-Show', 'News', 'Reality-TV', 'Short', 'Film-Noir']

    for name in weghalen:
        del movie_data[name]

    #print(movie_data)
    return movie_data

movie_data_1 = del_sparse_cols(movie_data_1)
movie_data_2 = del_sparse_cols(movie_data_2)
movie_data_3 = del_sparse_cols(movie_data_3)
movie_data_4 = del_sparse_cols(movie_data_4)

In [47]:
def plot_cor(data):
    plt.figure(figsize=(10,10), tight_layout=True)
    plt.pcolor(data.corr())
    plt.yticks(np.arange(0.5, len(list(data)), 1), list(data))
    plt.xticks(np.arange(0.5, len(list(data)), 1), list(data), rotation=90)
    plt.show()

#plot_cor(movie_data_4)

# Trainset en testset

In [48]:
def split_to_sets(movie_data):
    movie_data_sets = {'movie_training': 0, 'ratings_training': 0, 
                       'movie_validation': 0, 'ratings_validation': 0, 
                       'movie_test': 0, 'ratings_test': 0}
    
    ratings = movie_data['imdb_score'].values
    del movie_data['imdb_score']

    X = movie_data.values
    X_std = StandardScaler().fit_transform(X)

    number_of_samples = len(ratings)
    np.random.seed(0)
    random_indices = np.random.permutation(number_of_samples)
    num_training_samples = round(number_of_samples*0.6)
    num_validation_samples = round(number_of_samples*0.2)+num_training_samples

    movie_data_sets['movie_training'] = X_std[random_indices[:num_training_samples]]
    ratings_training = ratings[random_indices[:num_training_samples]]

    movie_data_sets['movie_validation'] = X_std[random_indices[num_training_samples:num_validation_samples]]
    movie_data_sets['ratings_validation'] = ratings[random_indices[num_training_samples:num_validation_samples]]

    movie_data_sets['movie_test'] = X_std[random_indices[num_validation_samples:]]
    movie_data_sets['ratings_test'] = ratings[random_indices[num_validation_samples:]]

    movie_data_sets['ratings_training'] = list(ratings_training)

    print(len(movie_data_sets['ratings_training']))
    print(len(movie_data_sets['ratings_validation']))
    print(len(movie_data_sets['ratings_test']))
    
    return movie_data_sets

sets = []
sets.append(split_to_sets(movie_data_1))
sets.append(split_to_sets(movie_data_2))
sets.append(split_to_sets(movie_data_3))
sets.append(split_to_sets(movie_data_4))

2263
754
754
2694
898
898
2866
955
956
2940
980
980


# Leren

In [49]:
def relative_error(y_predict, y):
    
    error = 0

    for i in range(len(y)):
        error += (abs(y_predict[i]-y[i]))/y[i]
    training_error = error/len(y)*100
    
    return training_error

In [50]:
def analyse_predictions(regr, sets, statdict, doprint=False):
    x_test = sets['movie_test']
    y_test = sets['ratings_test']
    
    regr.fit (sets['movie_training'], sets['ratings_training'])

    prediction = regr.predict(x_test)
    results_list.append(prediction)

    # The mean squared error
    print("Mean squared error: %.2f"
          % np.mean((prediction - y_test) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % regr.score(x_test, y_test))

    
    
    predres = list(regr.predict(x_test))
    actual = list(y_test)
    highest = avg = error = 0
    lowest = predres[0]
    for i in range(len(predres)):
        diff = abs(predres[i] - actual[i])
        avg += diff
        error += abs(predres[i] - actual[i]) / actual[i]
        
        if doprint:
            print("predicted:", predres[i]) 
            print ("\tActual:", actual[i])
            print ("\tDifference:", abs(predres[i] - actual[i]))

        if diff > highest:
            highest = diff
        if diff < lowest:
            lowest = diff

    errper = error / len(x_test) * 100
    avgdif =  avg / len(predres)
    
    if doprint:
        #print "Number of Samples:", len(predres)
        #print "Highest difference:", highest
        #print "Lowest difference:", lowest
        #print "average difference:", avgdif
        print("Error Percentage:", errper)
    
#     statdict['highest'].append(highest)
#     statdict['lowest'].append(lowest)
#     statdict['avgdif'].append(avgdif)
    statdict['errper'].append(errper)
    
    return statdict




results_list = []
statdicts = []
for i in range(4):
    statdict = {'names':[], 'errper': [], 'avgdif': [], 'highest': [], 'lowest': [], 'predictions': []}
    statdicts.append(statdict)

### neural networks

In [51]:
from sklearn.neural_network import MLPRegressor

def neural(statdict, sets):

    neural = MLPRegressor(hidden_layer_sizes =(100), activation = 'logistic', solver = 'adam', max_iter = 1000)

    neural.fit(sets['movie_training'], sets['ratings_training'])
    y_neural_train = neural.predict(sets['movie_training'])
    y_neural_test = neural.predict(sets['movie_test'])

    training_error = relative_error(y_neural_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(training_error)+" percent"+" in neural network algorithm")

    test_error = relative_error(y_neural_test,sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in neural network algorithm")
    
    statdict['errper'].append(test_error)
    statdict['names'].append('neural')
    statdict['predictions'].append(y_neural_test)
    
    return statdict

for i in range(4):
    statdicts[i] = neural(statdicts[i], sets[i])

Train error = 8.299093859692332 percent in neural network algorithm
Test error = 9.836597018707897 percent in neural network algorithm
Train error = 8.836540395985878 percent in neural network algorithm
Test error = 10.728500585657429 percent in neural network algorithm
Train error = 10.687027307058674 percent in neural network algorithm
Test error = 12.566443550420809 percent in neural network algorithm
Train error = 7.564529533405169 percent in neural network algorithm
Test error = 10.998149432880453 percent in neural network algorithm


### k-nearest neighbors

In [52]:
def do_knn(statdict, sets, n_neighbors=5, weights='uniform'):
        
    neighbors = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
    neighbors.fit(sets['movie_training'], sets['ratings_training'])
    y_neighbors_train = neighbors.predict(sets['movie_training'])
    #y_predict_train = list(y_predict_train)

    train_error = relative_error(y_neighbors_train,sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in knn algorithm")

    y_neighbors_test = neighbors.predict(sets['movie_test'])
    #y_predict_test = list(y_predict_test)

    test_error = relative_error(y_neighbors_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

    statdict['errper'].append(test_error)
    statdict['names'].append('kNN')
    statdict['predictions'].append(y_neighbors_test)
    
    return statdict

for i in range(4):
    statdicts[i] = do_knn(statdicts[i], sets[i], n_neighbors=15, weights='distance')

Train error = 0.0 percent in knn algorithm
Test error = 11.049064079622106 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 11.769553886564413 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 13.31806282108924 percent in knn algorithm
Train error = 0.0 percent in knn algorithm
Test error = 11.878901778400039 percent in knn algorithm


### Support vector machine

In [53]:
from sklearn.svm import SVR, NuSVR, LinearSVR

def do_svr(statdict, sets):
    svr = SVR()
    svr.fit(sets['movie_training'], sets['ratings_training'])
    y_svr_train = svr.predict(sets['movie_training'])
    #y_train = list(y_train)

    train_error = relative_error(y_svr_train, sets['ratings_training'])

    print ("Train error = "+'{}'.format(train_error))

    y_svr_test = svr.predict(sets['movie_test'])
    #y_test = list(y_test)

    test_error = relative_error(y_svr_test, sets['ratings_test'])

    print ("Test error = "+'{}'.format(test_error))

    statdict['errper'].append(test_error)
    statdict['names'].append('SVM')
    statdict['predictions'].append(y_svr_test)
    
    return statdict

for i in range(4):
    statdicts[i] = do_svr(statdicts[i], sets[i])

Train error = 8.187619393138535
Test error = 10.250489491713079
Train error = 8.615541896633948
Test error = 10.91478683155302
Train error = 8.697265271348568
Test error = 12.180230344638748
Train error = 9.150128853631955
Test error = 10.858612788391126


### Decision trees

In [54]:
from sklearn.tree import DecisionTreeRegressor

def do_dt(statdict, sets):
    trees = DecisionTreeRegressor()
    trees.fit(sets['movie_training'], sets['ratings_training'])
    y_trees_train = trees.predict(sets['movie_training'])

    train_error = relative_error(y_trees_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in decision trees")

    y_trees_test = trees.predict(sets['movie_test'])

    test_error = relative_error(y_trees_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in decision trees")

    statdict['errper'].append(test_error)
    statdict['names'].append('DT')
    statdict['predictions'].append(y_trees_test)
    
    return statdict

for i in range(4):
    statdicts[i] = do_dt(statdicts[i], sets[i])

Train error = 4.749036234303192e-16 percent in decision trees
Test error = 12.594576574137022 percent in decision trees
Train error = 3.8335588854045244e-16 percent in decision trees
Test error = 13.019590560866554 percent in decision trees
Train error = 3.859605849767512e-16 percent in decision trees
Test error = 15.274257462941554 percent in decision trees
Train error = 3.464430015449405e-16 percent in decision trees
Test error = 14.62629791711067 percent in decision trees


### Random Forest

In [55]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor

def do_randforest(statdict, sets):
    random_forest = RandomForestRegressor(n_estimators = 20)
    random_forest.fit(sets['movie_training'], sets['ratings_training'])
    y_forest_train = random_forest.predict(sets['movie_training'])

    train_error = relative_error(y_forest_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_forest_test = random_forest.predict(sets['movie_test'])

    test_error = relative_error(y_forest_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('Rfor')
    statdict['predictions'].append(y_forest_test)
    
    return statdict

for i in range(4):
    statdicts[i] = do_randforest(statdicts[i], sets[i])

Train error = 3.8075889505240545 percent in random forest
Test error = 9.483932555437839 percent in random forest
Train error = 3.8133884867744183 percent in random forest
Test error = 10.483396158004812 percent in random forest
Train error = 3.845012462176091 percent in random forest
Test error = 11.259504979854526 percent in random forest
Train error = 4.084538519059716 percent in random forest
Test error = 10.575777785653967 percent in random forest


### Extremely randomized trees

In [56]:
def do_xrandtrees(statdict, sets):
    extra = ExtraTreesRegressor(n_estimators = 20)
    extra.fit(sets['movie_training'], sets['ratings_training'])
    y_extra_train = extra.predict(sets['movie_training'])

    train_error = relative_error(y_extra_train, sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_extra_test = extra.predict(sets['movie_test'])

    test_error = relative_error(y_extra_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('XRfor')
    statdict['predictions'].append(y_extra_test)
    
    return statdict

for i in range(4):
    statdicts[i] = do_xrandtrees(statdicts[i], sets[i])

Train error = 1.7886980415022033e-14 percent in random forest
Test error = 9.879029757442733 percent in random forest
Train error = 1.7894243174509108e-14 percent in random forest
Test error = 10.482494151259939 percent in random forest
Train error = 1.7990798878596814e-14 percent in random forest
Test error = 11.76399273598605 percent in random forest
Train error = 1.821391130462263e-14 percent in random forest
Test error = 10.573558399131432 percent in random forest


### Bagging regressor

In [57]:
def do_baggingregr(statdict, sets):
    #regressor  = BaggingRegressor(MLPRegressor(max_iter = 300))
    bagging = BaggingRegressor(RandomForestRegressor())
    bagging.fit(sets['movie_training'], sets['ratings_training'])
    y_bagging_train = bagging.predict(sets['movie_training'])
    #y_predict_train = list(y_predict_train)

    train_error = relative_error(y_bagging_train ,sets['ratings_training'])

    print("Train error = "+'{}'.format(train_error)+" percent"+" in random forest")

    y_bagging_test = bagging.predict(sets['movie_test'])
    #y_predict_test = list(y_predict_test)

    test_error = relative_error(y_bagging_test, sets['ratings_test'])

    print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

    statdict['errper'].append(test_error)
    statdict['names'].append('bag')
    statdict['predictions'].append(y_bagging_test)
    
    return statdict

# for i in range(4):
#     statdicts[i] = do_baggingregr(statdicts[i], sets[i])

# Validation

### neural networks

In [58]:
# neural_1 = MLPRegressor(hidden_layer_sizes =(100), activation = 'logistic', solver = 'adam', max_iter = 1000)
# neural_2 = MLPRegressor(hidden_layer_sizes =(50), activation = 'logistic', solver = 'adam', max_iter = 1000)
# neural_3 = MLPRegressor(hidden_layer_sizes =(10), activation = 'logistic', solver = 'adam', max_iter = 1000)
# neural_4 = MLPRegressor(hidden_layer_sizes =(200), activation = 'logistic', solver = 'adam', max_iter = 1000)

# neural_1.fit(movie_training, ratings_training)
# neural_2.fit(movie_training, ratings_training)
# neural_3.fit(movie_training, ratings_training)
# neural_4.fit(movie_training, ratings_training)

# y_1_neural = neural_1.predict(movie_validation)
# y_2_neural = neural_2.predict(movie_validation)
# y_3_neural = neural_3.predict(movie_validation)
# y_4_neural = neural_4.predict(movie_validation)

# error_1_neural = relative_error(y_1_neural, ratings_validation)
# error_2_neural = relative_error(y_2_neural, ratings_validation)
# error_3_neural = relative_error(y_3_neural, ratings_validation)
# error_4_neural = relative_error(y_4_neural, ratings_validation)

# print(error_1_neural)
# print(error_2_neural)
# print(error_3_neural)
# print(error_4_neural)

### Support vector machines

In [59]:
# de parameters die kunnen worden aangepast 

# svr_1 = SVR(C = 0.0001)
# svr_2 = SVR(C = 0.01)
# svr_3 = SVR(C = 1.0)
# svr_4 = SVR(C = 100)
# svr_5 = SVR(C = 10000)

# svr_1 = SVR(epsilon = 0.001)
# svr_2 = SVR(epsilon = 0.01)
# svr_3 = SVR(epsilon = 0.1)
# svr_4 = SVR(epsilon = 10)
# svr_5 = SVR(epsilon = 100)

# svr_1.fit(movie_training, ratings_training)
# svr_2.fit(movie_training, ratings_training)
# svr_3.fit(movie_training, ratings_training)
# svr_4.fit(movie_training, ratings_training)
# svr_5.fit(movie_training, ratings_training)

# y_1_svr = svr_1.predict(movie_validation)
# y_2_svr = svr_2.predict(movie_validation)
# y_3_svr = svr_3.predict(movie_validation)
# y_4_svr = svr_4.predict(movie_validation)
# y_5_svr = svr_5.predict(movie_validation)

# error_1_svr = relative_error(y_1_svr, ratings_validation)
# error_2_svr = relative_error(y_2_svr, ratings_validation)
# error_3_svr = relative_error(y_3_svr, ratings_validation)
# error_4_svr = relative_error(y_4_svr, ratings_validation)
# error_5_svr = relative_error(y_5_svr, ratings_validation)

# print(error_1_svr)
# print(error_2_svr)
# print(error_3_svr)
# print(error_4_svr)
# print(error_5_svr)

### Random forest

In [60]:
# forest_1 = RandomForestRegressor(n_estimators = 5)
# forest_2 = RandomForestRegressor(n_estimators = 10)
# forest_3 = RandomForestRegressor(n_estimators = 20)
# forest_4 = RandomForestRegressor(n_estimators = 40)
# forest_5 = RandomForestRegressor(n_estimators = 80)

# forest_1.fit(movie_training, ratings_training)
# forest_2.fit(movie_training, ratings_training)
# forest_3.fit(movie_training, ratings_training)
# forest_4.fit(movie_training, ratings_training)
# forest_5.fit(movie_training, ratings_training)

# y_1_forest = forest_1.predict(movie_validation)
# y_2_forest = forest_2.predict(movie_validation)
# y_3_forest = forest_3.predict(movie_validation)
# y_4_forest = forest_4.predict(movie_validation)
# y_5_forest = forest_5.predict(movie_validation)

# error_1_forest = relative_error(y_1_forest, ratings_validation)
# error_2_forest = relative_error(y_2_forest, ratings_validation)
# error_3_forest = relative_error(y_3_forest, ratings_validation)
# error_4_forest = relative_error(y_4_forest, ratings_validation)
# error_5_forest = relative_error(y_5_forest, ratings_validation)

# print(error_1_forest)
# print(error_2_forest)
# print(error_3_forest)
# print(error_4_forest)
# print(error_5_forest)

### Extremely randomized trees

In [61]:
# extra_1 = ExtraTreesRegressor(n_estimators = 5)
# extra_2 = ExtraTreesRegressor(n_estimators = 10)
# extra_3 = ExtraTreesRegressor(n_estimators = 20)
# extra_4 = ExtraTreesRegressor(n_estimators = 40)
# extra_5 = ExtraTreesRegressor(n_estimators = 80)

# # extra_1 = ExtraTreesRegressor(n_estimators = 20, max_features = 'auto')
# # extra_2 = ExtraTreesRegressor(n_estimators = 20, max_features = 'sqrt')
# # extra_3 = ExtraTreesRegressor(n_estimators = 20, max_features = 'log2')
# # extra_4 = ExtraTreesRegressor()
# # extra_5 = ExtraTreesRegressor()

# extra_1.fit(movie_training, ratings_training)
# extra_2.fit(movie_training, ratings_training)
# extra_3.fit(movie_training, ratings_training)
# extra_4.fit(movie_training, ratings_training)
# extra_5.fit(movie_training, ratings_training)

# y_1_extra = extra_1.predict(movie_validation)
# y_2_extra = extra_2.predict(movie_validation)
# y_3_extra = extra_3.predict(movie_validation)
# y_4_extra = extra_4.predict(movie_validation)
# y_5_extra = extra_5.predict(movie_validation)

# error_1_extra = relative_error(y_1_extra, ratings_validation)
# error_2_extra = relative_error(y_2_extra, ratings_validation)
# error_3_extra = relative_error(y_3_extra, ratings_validation)
# error_4_extra = relative_error(y_4_extra, ratings_validation)
# error_5_extra = relative_error(y_5_extra, ratings_validation)

# print(error_1_extra)
# print(error_2_extra)
# print(error_3_extra)
# print(error_4_extra)
# print(error_5_extra)

### k-nearest neighbor

In [62]:
# neighbor_1 = KNeighborsRegressor()


### Decision trees

In [63]:
# trees_1 =

# Ensemble

### Neural networks and svr and random forest

In [64]:
def do_ensemble(statdict, sets):
    y_svr_test = statdict['predictions'][statdict['names'].index('SVM')]
    y_forest_test = statdict['predictions'][statdict['names'].index('Rfor')]
    y_extra_test = statdict['predictions'][statdict['names'].index('XRfor')]
    ratings_test = sets['ratings_test']


    prediction = [sum(x)/3 for x in zip(y_svr_test, y_forest_test, y_extra_test)]

    counter = 0

    for i in range(len(prediction)):
        if abs(prediction[i]-ratings_test[i])>1.0:
            counter+= 1
            #print(prediction[i], ratings_test[i])

    error_ensemble = relative_error(prediction, ratings_test)

    statdict['errper'].append(error_ensemble)
    statdict['names'].append('ensemble')
    statdict['predictions'].append(prediction)

    print(error_ensemble)
    print(counter/980.0)
    
    return statdict

for i in range(4):
    statdicts[i] = do_ensemble(statdicts[i], sets[i])

9.22508460741
0.09795918367346938
10.0606737998
0.12959183673469388
11.2156097336
0.1510204081632653
10.116869678
0.1673469387755102


In [65]:
# print(movie_data.columns)

In [66]:
# movie_data['color'].describe()

In [67]:
from sklearn import linear_model

def do_linear_models(statdict, sets):
    regr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('linreg')


    regr = linear_model.Ridge()
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('ridgereg')


    regr = linear_model.Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000000,
       normalize=False, positive=False, precompute=False, random_state=None,
       selection='cyclic', tol=0.0001, warm_start=False)
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('lassoreg')


    regr = linear_model.BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
           fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
           normalize=False, tol=0.001, verbose=False)
    statdict = analyse_predictions(regr, sets, statdict)
    statdict['names'].append('bayesian')
    
    return statdict


for i in range(4):
    statdicts[i] = do_linear_models(statdicts[i], sets[i])

Mean squared error: 0.69
Variance score: 0.37
Mean squared error: 0.69
Variance score: 0.38
Mean squared error: 0.68
Variance score: 0.39
Mean squared error: 0.67
Variance score: 0.39
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.72
Variance score: 0.41
Mean squared error: 0.87
Variance score: 0.34
Mean squared error: 0.87
Variance score: 0.33
Mean squared error: 0.88
Variance score: 0.33
Mean squared error: 0.88
Variance score: 0.33
Mean squared error: 0.77
Variance score: 0.38
Mean squared error: 0.77
Variance score: 0.38
Mean squared error: 0.78
Variance score: 0.38
Mean squared error: 0.78
Variance score: 0.38


In [70]:
def plot_threshold(statdicts):
    n_groups = len(statdicts[0]['names'])

    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    
    index = np.arange(n_groups)
    bar_width = 0.9 * (1.0 / 4)

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, statdicts[0]['errper'], bar_width,
                     alpha=opacity,
                     color='r',
                     error_kw=error_config,
                     label="threshold 1")

    rects2 = plt.bar(index + bar_width, statdicts[1]['errper'], bar_width,
                     alpha=opacity,
                     color='g',
                     error_kw=error_config,
                     label="threshold 2")

    rects3 = plt.bar(index + bar_width * 2, statdicts[2]['errper'], bar_width,
                     alpha=opacity,
                     color='b',
                     error_kw=error_config,
                     label="threshold 3")

    rects4 = plt.bar(index + bar_width * 3, statdicts[3]['errper'], bar_width,
                     alpha=opacity,
                     color='c',
                     error_kw=error_config,
                     label="threshold 4")

    plt.xlabel('Algorithm')
    plt.ylabel('Error Percentages (in %)')
    plt.title('Error Percentages for various algorithms depending on the threshold')
    plt.xticks(index + bar_width * 2, statdict['names'], rotation=45)
    plt.tight_layout()
    plt.legend(bbox_to_anchor=(0, 0.3), loc=2, borderaxespad=0.)
    plt.show()
    
plot_threshold(statdicts)

In [69]:
def plot_errors(statdict):
    n_groups = len(statdict['names'])

    fig, ax = plt.subplots()
    
    index = np.arange(n_groups)
    bar_width = 0.9

    opacity = 0.4
    error_config = {'ecolor': '0.3'}

    rects1 = plt.bar(index, statdict['errper'], bar_width,
                     alpha=opacity,
                     color='r',
                     error_kw=error_config)

    plt.xlabel('Algorithm')
    plt.ylabel('Error Percentages (in %)')
    plt.title('Error Percentages for various algorithms using threshold 4')
    plt.xticks(index + bar_width * 0.5, statdict['names'], rotation=90)
    plt.tight_layout()
    plt.show()

plot_errors(statdicts[3])